In [ ]:
!pip install sqlalchemy 
!pip install pandas
!pip install pyodbc
!pip install matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sqlalchemy import create_engine 

In [ ]:
engine = create_engine("mssql+pyodbc://LAPTOP-09HP798D/AdventureWorks2022?trusted_connection=yes&driver=ODBC+Driver+17+For+SQL+Server")

In [ ]:
connection = engine.connect()

In [ ]:
# -- 00100 -- Totalt antal produkter

df=pd.read_sql(sql='''

SELECT COUNT(*) AS [Totalt Antal Produkter]
FROM [AdventureWorks2022].[Production].[Product];

    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00110 -- Totalt Försäljnings Belopp

df=pd.read_sql(sql='''

SELECT CAST(SUM([TotalDue]) AS INT) AS [Totalt Försäljnings Belopp]
FROM [AdventureWorks2022].[Sales].[SalesOrderHeader];

    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00120 -- Genomsnittligt Ordervärde

df=pd.read_sql(sql='''

SELECT CAST(AVG([TotalDue]) AS decimal(10,2)) AS [Genomsnittligt Ordervärde]
FROM [AdventureWorks2022].[Sales].[SalesOrderHeader];

    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00130 -- Totalt Spenderat Belopp

df=pd.read_sql(sql='''

SELECT TOP 10 [CustomerID], CAST(SUM([TotalDue]) AS int) AS [Totalt Spenderat Belopp]
FROM [AdventureWorks2022].[Sales].[SalesOrderHeader]
GROUP BY [CustomerID] ORDER BY [Totalt Spenderat Belopp] DESC;

    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00140 -- Företagets personal 

df=pd.read_sql(sql='''

SELECT COUNT([JobTitle]), [JobTitle]  
FROM [AdventureWorks2022].[humanresources].[employee]
GROUP BY [JobTitle]
ORDER BY [JobTitle];

    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00150 -- Antal och Kategorier av Personer i Företaget

df=pd.read_sql(sql='''

SELECT COUNT([PersonType]) AS [Antal], 
	CASE
		WHEN [persontype] = 'SC' THEN 'Store Contact'
		WHEN [persontype] = 'IN' THEN 'Individual (retail) customer'
		WHEN [persontype] = 'SP' THEN 'Sales person'
		WHEN [persontype] = 'EM' THEN 'Employee (non-sales)'
		WHEN [persontype] = 'VC' THEN 'Vendor contact'
		WHEN [persontype] = 'GC' THEN 'General contact'
	END AS [Kategori]
FROM [AdventureWorks2022].[Person].[Person]
GROUP BY [persontype];

    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00160 -- Avdelningar och antal anställda

df=pd.read_sql(sql='''

SELECT 
	A.[Name] AS [Avdelning], 
	COUNT(*) AS [Antal Anställda]
FROM [AdventureWorks2022].[HumanResources].[Department] AS A
JOIN [AdventureWorks2022].[HumanResources].[EmployeeDepartmentHistory] AS B 
ON A.[DepartmentID] = B.[DepartmentID]
GROUP BY A.[Name];

    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00170 -- Försäljning per produktkategori

df=pd.read_sql(sql='''

SELECT 
	E.[Name] AS [Produktkategori],
	CAST(SUM(B.[LineTotal]) AS INT) AS [Total delsumma]
FROM 
	[AdventureWorks2022].[Sales].[SalesOrderDetail] AS B 
	LEFT JOIN Production.Product AS C ON B.[ProductID] = C.[ProductID]
	JOIN Production.ProductSubcategory AS D ON C.[ProductSubcategoryID] = D.[ProductSubcategoryID]
	JOIN Production.ProductCategory AS E ON D.[ProductCategoryID] = E.[ProductCategoryID]
GROUP BY 
	E.[Name];

    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00200 -- Totalt antal kunder

df=pd.read_sql(sql= '''

SELECT 
	COUNT(DISTINCT [CustomerID]) AS [Totalt Antal Kunder]
FROM 
	[AdventureWorks2022].[Sales].[SalesOrderHeader];

    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00300 -- Kundfördelning per Land

df=pd.read_sql(sql= '''

SELECT 
	T.[CountryRegionCode] AS [Land],
	COUNT(DISTINCT S.[CustomerID]) AS [Antal kunder]
FROM 
	[Sales].[SalesOrderHeader] AS S
    LEFT OUTER JOIN [Sales].[SalesTerritory] AS T
    ON S.[TerritoryID] = T.[TerritoryID]
GROUP BY 
    T.[CountryRegionCode]
ORDER BY 
	[Land];
    
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00400 -- Totalt antal beställningar 

df=pd.read_sql(sql= '''

SELECT 
	COUNT([SalesOrderID]) AS [Totalt Antal Beställningar]
FROM 
	[AdventureWorks2022].[Sales].[SalesOrderHeader];
        
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# Totalt antal kunder i varje land delat efter år

df=pd.read_sql(sql= '''

SELECT 
	YEAR(S.[OrderDate]) AS [År],
	T.[CountryRegionCode] AS [Landskod],
	COUNT(DISTINCT [CustomerID]) AS [Antal kunder]
FROM 
	[Sales].[SalesOrderHeader] AS S
    LEFT OUTER JOIN [Sales].[SalesTerritory] AS T
    ON S.[TerritoryID] = T.[TerritoryID]
GROUP BY 
    YEAR(S.[OrderDate]), T.[CountryRegionCode]
ORDER BY 
	[År], [Landskod];
    
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00500 -- Antal beställningar per år

df=pd.read_sql(sql= '''

SELECT 
	YEAR([OrderDate]) AS [År],
	COUNT([SalesOrderID]) AS [Antal Beställningar]
FROM 
	[Sales].[SalesOrderHeader]
--	  WHERE YEAR([OrderDate]) IN ('2012', '2013')
GROUP BY 
    YEAR([OrderDate])
ORDER BY 
	YEAR([OrderDate]);
    
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00600 -- Antal beställningar per kvartal

df=pd.read_sql(sql= '''

SELECT 
    YEAR([OrderDate]) AS [År],
    DATEPART(QUARTER, [OrderDate]) AS [Kvartal],
    COUNT([SalesOrderID]) AS [Antal Beställningar]
FROM 
    [Sales].[SalesOrderHeader]
GROUP BY 
    YEAR([OrderDate]), DATEPART(QUARTER, [OrderDate])
ORDER BY 
    [År], [Kvartal]
        
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00700 -- Antal beställningar per månad

df=pd.read_sql(sql= '''

SELECT 
	YEAR([OrderDate]) AS [År],
	MONTH([OrderDate]) AS [Månad],
	COUNT(SalesOrderID) AS [Ordrar]
FROM 
	[AdventureWorks2022].[Sales].[SalesOrderHeader]
GROUP BY 
	YEAR([OrderDate]), MONTH([OrderDate])
ORDER BY 
	[År], [Månad] ASC;
        
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00800 -- Antal beställningar per land

df=pd.read_sql(sql= '''

SELECT 
	T.[CountryRegionCode] AS [Land],
	COUNT(S.[CustomerID]) AS [Antal beställningar]
FROM 
	[AdventureWorks2022].[Sales].[SalesOrderHeader] AS S
    LEFT OUTER JOIN [Sales].[SalesTerritory] AS T
    ON S.[TerritoryID] = T.[TerritoryID]
GROUP BY 
    T.[CountryRegionCode]
ORDER BY 
	[Land];
        
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 00900 -- Antal beställningar per land och år

df=pd.read_sql(sql= '''

SELECT 
	YEAR(S.[OrderDate]) AS [År],
	T.[CountryRegionCode] AS [Land],
	COUNT(S.[CustomerID]) AS [Antal Beställningar]
FROM 
	[AdventureWorks2022].[Sales].[SalesOrderHeader] AS S
    LEFT OUTER JOIN [Sales].[SalesTerritory] AS T
    ON S.[TerritoryID] = T.[TerritoryID]
GROUP BY 
    YEAR(S.[OrderDate]), T.[CountryRegionCode]
ORDER BY 
	[År], [Land];
            
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- xxxxx -- Plottning av antal beställningar per år (2012 0ch 2013) och land

df=pd.read_sql(sql= '''

SELECT 
	YEAR(S.[OrderDate]) AS [År],
	T.[CountryRegionCode] AS [Land],
	COUNT(S.[CustomerID]) AS [Antal Beställningar]
FROM 
	[Sales].[SalesOrderHeader] AS S
    LEFT OUTER JOIN [Sales].[SalesTerritory] AS T
    ON S.[TerritoryID] = T.[TerritoryID]
    WHERE
    YEAR(S.[OrderDate]) IN (2012, 2013)
GROUP BY 
    YEAR(S.[OrderDate]), T.[CountryRegionCode]
ORDER BY 
	[År], [Land];
            
    ''', con=connection)

pivot_df = df.pivot(index='Land', columns='År', values='Antal Beställningar')
pivot_df.plot(kind='bar', figsize=(8, 3), rot=0)
plt.title('Antal beställningar per år och land')
plt.xlabel('Land')
plt.ylabel('Antal Beställningar')
plt.legend(title='År')
plt.show()

In [ ]:
# -- 01000 -- Fördelning av antal beställningar per kund

df=pd.read_sql(sql= '''

WITH [CTE_AntalBeställningarPerKund] AS 
	(
		SELECT 
			COUNT([SalesOrderID]) AS [AntalBeställningarPerKund]
		FROM 
			[AdventureWorks2022].[Sales].[SalesOrderHeader]
		GROUP BY 
			[CustomerID]
	)

SELECT 
	[AntalBeställningarPerKund] AS [Antal beställningar per kund],
	COUNT([AntalBeställningarPerKund]) AS [Antal kunder]
FROM 
	[CTE_AntalBeställningarPerKund]
GROUP BY
	[AntalBeställningarPerKund]
ORDER BY 
	[AntalBeställningarPerKund] DESC;

    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- xxxxx -- Plottning av fördelningen av antal beställningar per kund

plt.plot(df['Antal beställningar per kund'], df['Antal kunder'], marker='o')

plt.xticks(df['Antal beställningar per kund'])
plt.xlabel('Antal beställningar per kund')
plt.ylabel('Antal kunder')
plt.title('Fördelning av Antal Beställningar Per Kund')

plt.grid(True)
plt.show()


In [ ]:
# -- 02000 -- Årlig försäljningssammanfattning

df = pd.read_sql(sql= """

WITH [CTE_ÅrTotal] AS 
    (
    SELECT 
        YEAR([OrderDate]) AS [OrderDate], 
        COUNT([SalesOrderID]) AS [TotaltAntalBeställningar],
        SUM([SubTotal]) AS [DelSumma],
        SUM([TaxAmt]) AS [Skatt],
        SUM([Freight]) AS [Frakt],
        SUM([TotalDue]) AS [TotaltBelopp]
    FROM 
        [AdventureWorks2022].[Sales].[SalesOrderHeader]
    GROUP BY 
        YEAR([OrderDate])
    )

SELECT 
    [OrderDate] AS [År], 
    [TotaltAntalBeställningar] AS [Antal Beställningar],
    CAST(ROUND([DelSumma], 0) AS int) AS [Del Summa], 
    [Procentandel DelSumma] = CAST(ROUND([DelSumma] * 100.0 / SUM([TotaltBelopp]) OVER (PARTITION BY [OrderDate]), 2) AS decimal(5, 2)),
    CAST(ROUND([Skatt], 0) AS int) AS [Skatt Belopp], 
    [Procentandel Skatt] = CAST(ROUND([Skatt] * 100.0 / SUM([TotaltBelopp]) OVER (PARTITION BY [OrderDate]), 2) AS decimal(5, 2)),
    CAST(ROUND([Frakt], 0) AS int) AS [Frakt],
    [Procentandel Frakt] = CAST(ROUND([Frakt] * 100.0 / SUM([TotaltBelopp]) OVER (PARTITION BY [OrderDate]), 2) AS decimal(5, 2)),
    CAST(ROUND([TotaltBelopp], 0) AS int) AS [Totalt Belopp]
FROM 
    [CTE_ÅrTotal] 
ORDER BY 
    [OrderDate] ASC;
            
            """ , con=connection)

In [ ]:
display(df)

In [ ]:
# -- xxxxx -- Plottning av årlig Försäljningssammanfattning

df['År'] = df['År'].astype(str)

fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))

ax1.bar(df['År'], df['Del Summa'], width=0.6, color='dodgerblue', edgecolor='red', linewidth=2)
ax1.set_title('Del Summa')
ax1.set_xlabel('År')
ax1.set_ylabel('Del Summa (10 x Million $)')

ax2.bar(df['År'], df['Frakt'], width=0.6, color='gold', edgecolor='blue', linewidth=2)
ax2.set_title('Frakt')
ax2.set_xlabel('År')
ax2.set_ylabel('Frakt (10 x Million $)')

ax3.bar(df['År'], df['Skatt Belopp'], width=0.6, color='red', edgecolor='green', linewidth=2)
ax3.set_title('Skatt Belopp')
ax3.set_xlabel('År')
ax3.set_ylabel('Skatt Belopp (10 x Million $)')

plt.tight_layout()
plt.show()

In [ ]:
# -- xxxxx -- Plottning av totalt antal beställninga

df['År'] = df['År'].astype(str)

plt.figure(figsize=(4, 2))

plt.bar (df['År'], df['Antal Beställningar'], width=0.6, color='grey', edgecolor='orange', linewidth=2)
plt.title('Totalt antal beställningar per år')
plt.xlabel('År')
plt.ylabel('Totalt Antal Beställningar')

plt.show()

In [ ]:
# -- 03000 -- Försäljning per kvartal

df = pd.read_sql(sql= """

WITH [CTE_ÅrTotal] AS 
	(
    SELECT 
        YEAR([OrderDate]) AS [OrderDate], 
        DATEPART(QUARTER, [OrderDate]) AS [Quarter],
        COUNT([SalesOrderID]) AS [TotaltAntalBeställningar],
        SUM([SubTotal]) AS [DelSumma],
        SUM([TaxAmt]) AS [Skatt],
        SUM([Freight]) AS [Frakt],
        SUM([TotalDue]) AS [TotaltBelopp]
    FROM 
		[AdventureWorks2022].[Sales].[SalesOrderHeader]
    GROUP BY 
		YEAR([OrderDate]), DATEPART(QUARTER, [OrderDate])
	)

SELECT 
    [OrderDate] AS [År], 
    [Quarter] AS [Kvartal],
    [TotaltAntalBeställningar] AS [Antal Beställningar],

    CAST(ROUND([DelSumma], 0) AS int) AS [Del Summa], 
    [Procentandel DelSumma] = CAST(ROUND([DelSumma] * 100.0 / SUM([TotaltBelopp]) OVER (PARTITION BY [OrderDate], [Quarter]), 2) AS decimal(5, 2)),

    CAST(ROUND([Skatt], 0) AS int) AS [Skatt Belopp], 
    [Procentandel Skatt] = CAST(ROUND([Skatt] * 100.0 / SUM([TotaltBelopp]) OVER (PARTITION BY [OrderDate], [Quarter]), 2) AS decimal(5, 2)),

    CAST(ROUND([Frakt], 0) AS int) AS [Frakt],
    [Procentandel Frakt] = CAST(ROUND([Frakt] * 100.0 / SUM([TotaltBelopp]) OVER (PARTITION BY [OrderDate], [Quarter]), 2) AS decimal(5, 2)),

    CAST(ROUND([TotaltBelopp], 0) AS int) AS [Totalt Belopp]
FROM 
    [CTE_ÅrTotal] 
ORDER BY 
    [OrderDate], [Quarter] ASC;
            
            """ , con=connection)

In [ ]:
display(df)

In [ ]:
# -- 04000 -- Försäljning per månad 

df = pd.read_sql(sql= """

WITH [CTE_OrderTotalMonth] AS 
	(
    SELECT 
        YEAR([OrderDate]) AS [År],
		MONTH([OrderDate]) AS [Månad],
		COUNT(SalesOrderID) AS [Antal beställningar],
        SUM([TotalDue]) AS [Totalt belopp]
    FROM 
		[AdventureWorks2022].[Sales].[SalesOrderHeader]
    GROUP BY 
		YEAR([OrderDate]), MONTH([OrderDate])
	)

SELECT 
    [År], 
	[Månad],
	[Antal beställningar],
    CAST([Totalt belopp] AS int) AS [Totalt belopp]
FROM 
	[CTE_OrderTotalMonth]
ORDER BY
	[År] ASC, [Månad] ASC;
            
            """ , con=connection)

In [ ]:
display(df)

In [ ]:
# -- 05000 -- Månatlig MAX och MIN Totalt belopp

df = pd.read_sql(sql= """

WITH [CTE_MinMaxOrder] AS 
	(
    SELECT 
        YEAR([OrderDate]) AS [År],
		MONTH([OrderDate]) AS [Månad],
		COUNT(SalesOrderID) AS [Antal ordrar],
        SUM([TotalDue]) AS [Totalt belopp]
    FROM 
		[AdventureWorks2022].[Sales].[SalesOrderHeader]
    GROUP BY 
		YEAR([OrderDate]), MONTH([OrderDate])
	)

SELECT 
	[År], 
	[Månad], 
	[Antal ordrar], 
	CAST([Totalt belopp] AS INT) AS [Max/Min Totalt belopp]
FROM 
	[CTE_MinMaxOrder] 
WHERE 
	[Totalt belopp] = (SELECT MAX([Totalt belopp]) FROM [CTE_MinMaxOrder]) 
	OR 
	[Totalt belopp] = (SELECT MIN([Totalt belopp]) FROM [CTE_MinMaxOrder])
ORDER BY
	[Totalt belopp] DESC;
    
            """ , con=connection)

In [ ]:
display(df)

In [ ]:
# -- 06000 -- Medelvärdert för beställningar och totalt belopp, månadsvis

df = pd.read_sql(sql= """

SELECT 
    YEAR([OrderDate]) AS [År],
	MONTH([OrderDate]) AS [Månad],
	COUNT(SalesOrderID) AS [Antal beställningar],
    CAST(SUM([TotalDue]) AS INT) AS [Totalt belopp],
	CAST(AVG([TotalDue]) AS INT) AS [Medelvärde]
FROM 
	[AdventureWorks2022].[Sales].[SalesOrderHeader]
GROUP BY 
	YEAR([OrderDate]), MONTH([OrderDate])
ORDER BY 
	[Medelvärde]
        
            """ , con=connection)

In [ ]:
display(df)

In [ ]:
# -- 06110 -- Medelvärdet för alla försäljningsordrar i företaget 

df = pd.read_sql(sql= """

SELECT CAST(AVG([TotalDue]) AS INT) AS [Totalt Medelvärde]
FROM [AdventureWorks2022].[Sales].[SalesOrderHeader];
        
            """ , con=connection)

In [ ]:
display(df)

In [ ]:
# -- 07000 -- Kvartalsvis fördelning av online beställningar och beställningar av säljare

df = pd.read_sql(sql= """

WITH [CTE_OnlineOrderSP] AS
    (
	SELECT 
		YEAR([OrderDate]) AS [År],
		DATEPART(QUARTER, [OrderDate]) AS [Kvartal],
		SUM(CASE WHEN [OnlineOrderFlag] = 1 THEN 1 ELSE 0 END) AS [OnlineBeställningar],
		SUM(CASE WHEN [OnlineOrderFlag] = 0 THEN 1 ELSE 0 END) AS [Beställningar Av Säljare],
		COUNT(*) AS [Nämnare]
	FROM 
		[AdventureWorks2022].[Sales].[SalesOrderHeader]
	GROUP BY 
		YEAR([OrderDate]), DATEPART(QUARTER, [OrderDate])
    )

SELECT 
	[År],
	[Kvartal],
	[OnlineBeställningar] AS [Online Beställningar],
	[Beställningar Av Säljare] AS [Beställningar Av Säljare],
	CAST(100.0 * [OnlineBeställningar] / [Nämnare] AS decimal(3, 1)) AS [Andel: Online Beställning],
	CAST(100.0 * [Beställningar Av Säljare] / [Nämnare] AS decimal(3, 1)) AS [Andel: Beställningar av Säljare]
FROM
	[CTE_OnlineOrderSP]
ORDER BY 
	[År], [Kvartal];

            """ , con=connection)

In [ ]:
display(df)

In [ ]:
# Plottning av online beställningar och beställningar av säljare, kvartalvis

ax = df[['Kvartal', 'Online Beställningar', 'Beställningar Av Säljare']].plot(kind='bar', figsize=(7, 3), width=0.8)

ax.set_xlabel('Kvartal')
ax.set_ylabel('Antal Beställningar')
ax.set_title('Jämförelse av Onlinebeställningar och Beställningar av Säljare över Åren')
ax.legend(["Online Beställningar", "Beställningar Av Säljare"], loc='upper left')
ax.set_xticklabels(df['Kvartal'], rotation=0)
plt.show()



In [ ]:
# -- 08000 -- Försäljningsstatistik per Kvartal - Medelvärde och Standardavvikelse

df = pd.read_sql(sql= """

SELECT 
	YEAR([OrderDate]) AS [År], 
	DATEPART(QUARTER, [OrderDate]) AS [Kvartal],
	CAST(AVG([TotalDue]) AS int) AS [Genomsnitt Totalt belopp)],
	CAST(STDEV([TotalDue]) AS int) AS [Standardavvikelse Totalt belopp]
FROM 
	[AdventureWorks2022].[Sales].[SalesOrderHeader] 
GROUP BY  
	YEAR([OrderDate]), DATEPART(QUARTER, [OrderDate])
ORDER BY 
	[År], [Kvartal];

            """ , con=connection)

In [ ]:
display(df)

In [ ]:
# -- 09000 -- Median (50:e percentilen) av totalt belopp

df = pd.read_sql(sql= """

SELECT  DISTINCT
		PERCENTILE_CONT(0.5) 
		WITHIN GROUP (ORDER BY [TotalDue]) 
		OVER (PARTITION BY YEAR([OrderDate]), DATEPART(QUARTER, [OrderDate]))
		AS [Median Totalt belopp], 
		YEAR([OrderDate]) AS [År], 
		DATEPART(QUARTER, [OrderDate]) AS [Kvartal]
FROM 
		[AdventureWorks2022].[Sales].[SalesOrderHeader]
ORDER BY 
		[År], [Kvartal];

            """ , con=connection)

In [ ]:
display(df)

In [ ]:
# -- 01110 -- Antal beställningar och det totala försäljningsbeloppet per land för varje säljare

df=pd.read_sql(sql='''

SELECT 
	A.[SalesPersonID], 
	C.[FirstName], 
	C.[LastName], 
	D.[CountryRegionCode] AS [Land], 
	COUNT(A.[SalesPersonID]) AS [Antal Beställningar], 
	CAST(SUM(A.TotalDue) AS int) AS [Total Försäljning]
FROM 
	[AdventureWorks2022].[Sales].[SalesOrderHeader] AS A
	INNER JOIN [AdventureWorks2022].[Sales].[SalesPerson] AS B
	ON A.[SalesPersonID] = B.[BusinessEntityID]
	INNER JOIN [AdventureWorks2022].[Person].[Person] AS C
	ON B.[BusinessEntityID] = C.[BusinessEntityID]
	LEFT JOIN [AdventureWorks2022].[Sales].[SalesTerritory] AS D
	ON B.[TerritoryID] = D.[TerritoryID]
GROUP BY 
	A.[SalesPersonID], C.[FirstName], C.[LastName], D.[CountryRegionCode]
HAVING 
	A.[SalesPersonID] IS NOT NULL
ORDER BY 
	[Total Försäljning];
    
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 01120 -- Statistisk analys (konfidensintervall), Totalt

df=pd.read_sql(sql='''

WITH [CTE_konfidensintervall] AS 
	(
    SELECT 
        COUNT([TotalDue]) AS [n],
        CAST(AVG([TotalDue]) AS int) AS [Medelvärde],
        CAST(STDEV([TotalDue]) AS int) AS [Standardavvikelse]
    FROM 
        [AdventureWorks2022].[Sales].[SalesOrderHeader]
	) 

SELECT 
	[Medelvärde], 
	[Standardavvikelse],
	CAST(([Medelvärde]) - (1.96 * [Standardavvikelse] / SQRT(n)) AS INT) AS [KI_Nedre],
    CAST(([Medelvärde]) + (1.96 * [Standardavvikelse] / SQRT(n)) AS INT) AS [KI_Övre]
FROM 
	[CTE_konfidensintervall];
        
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 01121 -- Statistisk analys (Median), Totalt

df=pd.read_sql(sql='''

WITH [CTE_Median] AS 
	(
    SELECT TOP 50 PERCENT CAST([TotalDue] AS INT) AS [Median]
    FROM [AdventureWorks2022].[Sales].[SalesOrderHeader]
    ORDER BY [TotalDue] ASC
	)
SELECT TOP 1 [Median]
FROM CTE_Median ORDER BY [Median] DESC;
        
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 01122 -- Statistisk analys (konfidensintervall), Månadsvist

df=pd.read_sql(sql='''

SELECT 
    YEAR([OrderDate]) AS [År], 
	MONTH([OrderDate]) AS [Månad], 
	COUNT([SalesOrderID]) AS [Antal beställningar],
    CAST(AVG([TotalDue]) AS INT) AS [Medelvärde Totalt belopp],
    CAST(STDEV([TotalDue]) AS INT) AS [Standardavvikelse Totalt belopp],
    CAST((AVG([TotalDue]) - 1.96 * STDEV([TotalDue]) / SQRT(COUNT([TotalDue]))) AS INT) AS [KI_Nedre],
    CAST((AVG([TotalDue]) + 1.96 * STDEV([TotalDue]) / SQRT(COUNT([TotalDue]))) AS INT) AS [KI_Övre]
FROM 
    [AdventureWorks2022].[Sales].[SalesOrderHeader] 
GROUP BY  
    YEAR([OrderDate]), MONTH([OrderDate])
ORDER BY 
    [År], [Månad];
    
    ''', con=connection)

In [ ]:
display(df)

In [ ]:
# -- 01133 -- En funktion för att beräkna konfidensintervall för skillnaden mellan 
# medelvärden för två angivna år i tabellen: [AdventureWorks2022].[Sales].[SalesOrderHeader].

connection.execute( '''

CREATE FUNCTION dbo.SkillnadMedelvärde (@År1 INT, @År2 INT) 
RETURNS @SkillnadsIntervall TABLE (SkillnadÖvre INT, SkillnadNedre INT)
AS
BEGIN
    DECLARE @SkillnadÖvre INT;
    DECLARE @SkillnadNedre INT;

    SELECT @SkillnadÖvre = 
        ( -- (Medelvärde1 - Medelvärde2) +  z. sqrt [(standardavvickelse1)^2/n1 + (standardavvickelse2)^2/n2]
            ( -- Medelvärde1 - Medelvärde2
                CAST(AVG(CASE WHEN YEAR([OrderDate]) = @År1 THEN [TotalDue] END) AS INT)
                -
                CAST(AVG(CASE WHEN YEAR([OrderDate]) = @År2 THEN [TotalDue] END) AS INT)
            )
            +
            ( -- Z (1.96 för konfidensgrad 95 procent
                1.96
                *
                SQRT
                    (
                        ( -- (standardavvickelse1)^2/n1
                            POWER(STDEV(CASE WHEN YEAR([OrderDate]) = @År1 THEN [TotalDue] END), 2)
                            / NULLIF(COUNT(CASE WHEN YEAR([OrderDate]) = @År1 THEN [SalesOrderID] END), 0)
                        )
                        +
                        ( -- (standardavvickelse2)^2/n2
                            POWER(STDEV(CASE WHEN YEAR([OrderDate]) = @År2 THEN [TotalDue] END), 2)
                            / NULLIF(COUNT(CASE WHEN YEAR([OrderDate]) = @År2 THEN [SalesOrderID] END), 0)
                        )
                    )
            )
        )
    FROM [AdventureWorks2022].[Sales].[SalesOrderHeader]
    WHERE YEAR([OrderDate]) IN (@År1, @År2);

   
    SELECT @SkillnadNedre = 
        ( -- (Medelvärde1 - Medelvärde2) -  z. sqrt [(standardavvickelse1)^2/n1 + (standardavvickelse2)^2/n2]
            ( -- Medelvärde1 - Medelvärde2
                CAST(AVG(CASE WHEN YEAR([OrderDate]) = @År1 THEN [TotalDue] END) AS INT)
                -
                CAST(AVG(CASE WHEN YEAR([OrderDate]) = @År2 THEN [TotalDue] END) AS INT)
            )
            -
            ( -- Z (1.96 för konfidensgrad 95 procent
                1.96
                *
                SQRT
                    (
                        ( -- (standardavvickelse1)^2/n1
                            POWER(STDEV(CASE WHEN YEAR([OrderDate]) = @År1 THEN [TotalDue] END), 2)
                            / NULLIF(COUNT(CASE WHEN YEAR([OrderDate]) = @År1 THEN [SalesOrderID] END), 0)
                        )
                        +
                        ( -- (standardavvickelse2)^2/n2
                            POWER(STDEV(CASE WHEN YEAR([OrderDate]) = @År2 THEN [TotalDue] END), 2)
                            / NULLIF(COUNT(CASE WHEN YEAR([OrderDate]) = @År2 THEN [SalesOrderID] END), 0)
                        )
                    )
            )
        )
    FROM [AdventureWorks2022].[Sales].[SalesOrderHeader]
    WHERE YEAR([OrderDate]) IN (@År1, @År2);

	INSERT INTO @SkillnadsIntervall (SkillnadÖvre, SkillnadNedre) VALUES (@SkillnadÖvre, @SkillnadNedre)

    RETURN;
END;

    ''')

In [ ]:
df=pd.read_sql(sql='''
SELECT SkillnadÖvre, SkillnadNedre
FROM dbo.SkillnadMedelvärde(2012, 2013);
''', con=connection)

In [ ]:
display(df)